# Demo: Chuyển đổi Ngôn ngữ Tự nhiên thành Câu lệnh Phân quyền
### (NLP Cơ bản vs. Gemini LLM trên MongoDB)

Notebook này minh họa và so sánh hai phương pháp chuyển đổi yêu cầu ngôn ngữ tự nhiên thành câu lệnh phân quyền MongoDB:
1.  **Phương pháp NLP Cơ bản (Dựa trên Quy tắc):** Phân tích câu dựa trên các từ khóa và quy tắc đơn giản.
2.  **Phương pháp Dùng LLM (Gemini):** Sử dụng mô hình ngôn ngữ lớn Gemini để "hiểu" ngữ cảnh và sinh ra câu lệnh phức tạp hơn.

## Bước 1: Cài đặt, Kết nối CSDL và Tạo Dữ liệu Mẫu
Chúng ta sẽ tạo dữ liệu mẫu với cấu trúc phức tạp hơn để phù hợp với yêu cầu.

In [1]:
from pymongo import MongoClient
import json
import re
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Kết nối tới MongoDB server
MONGO_ADMIN_USER = 'mongo_user'
MONGO_ADMIN_PASSWORD = 'mongo_password'
MONGO_HOST = 'localhost'
MONGO_PORT = 27018

client = MongoClient(host=MONGO_HOST, port=MONGO_PORT, username=MONGO_ADMIN_USER, password=MONGO_ADMIN_PASSWORD)
db = client['access_control_demo_db']

# Cập nhật collection 'users' với các thuộc tính mới
db.users.drop()
users_collection = db.users
users_data = [
    {'id': 1, 'name': 'Alice', 'department': 'Engineering', 'projects': ['X'], 'security_training_completed': True},
    {'id': 2, 'name': 'Bob', 'department': 'Production', 'projects': [], 'security_training_completed': False},
    {'id': 3, 'name': 'Charlie', 'department': 'HR', 'projects': [], 'security_training_completed': True},
    {'id': 4, 'name': 'David', 'department': 'Sales', 'projects': ['Y'], 'security_training_completed': False},
    {'id': 5, 'name': 'Eve', 'department': 'Production', 'projects': ['X'], 'security_training_completed': True}
]
users_collection.insert_many(users_data)

print(f"Đã tạo xong dữ liệu mẫu trong CSDL '{db.name}'.")
print(f"Số lượng user: {users_collection.count_documents({})}")

Đã tạo xong dữ liệu mẫu trong CSDL 'access_control_demo_db'.
Số lượng user: 5


## Phần 2: Phương pháp sử dụng Gemini LLM

### Bước 2.1: Cấu hình và Khởi tạo Gemini
Lưu ý: Bạn cần có tệp `.env` chứa `GEMINI_API_KEY` để cell này hoạt động.

In [2]:
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
gemini_model = None

if not api_key:
    print("API Key của Gemini không được tìm thấy. Hãy chắc chắn bạn đã tạo file .env và thêm GEMINI_API_KEY vào đó.")
else:
    genai.configure(api_key=api_key)
    gemini_model = genai.GenerativeModel('gemini-2.5-flash')
    print("Đã khởi tạo Gemini Model thành công.")

Đã khởi tạo Gemini Model thành công.


### Bước 2.2: Thiết kế Prompt và Sinh câu lệnh

In [3]:
def generate_mongodb_command_with_gemini(natural_language_input):
    if not gemini_model:
        return "# Gemini model chưa được khởi tạo."

    # Prompt được thiết kế lại để tạo câu lệnh phân quyền
    prompt_lines = [
        "Bạn là một quản trị viên cơ sở dữ liệu MongoDB chuyên nghiệp. Nhiệm vụ của bạn là chuyển đổi một yêu cầu bằng ngôn ngữ tự nhiên thành một câu lệnh phân quyền MongoDB chính xác.",
        "Chỉ trả về câu lệnh MongoDB (ví dụ: db.createRole(...)), không thêm bất kỳ giải thích hay định dạng markdown nào.",
        "",
        "CSDL là 'access_control_demo_db'.",
        "Collection 'users' có cấu trúc: { 'id': number, 'name': string, 'department': string, 'projects': array, 'security_training_completed': boolean }",

        "Ví dụ (Phân quyền theo thuộc tính):",
           "Bây giờ, hãy chuyển đổi yêu cầu sau:",
        f"Yêu cầu: '{natural_language_input}'",
        "Câu lệnh:"
    ]
    prompt = "\n".join(prompt_lines)

    try:
        response = gemini_model.generate_content(prompt)
        command = response.text.strip()
        command = re.sub(r'^```(json|javascript)?\n|\n```$', '', command)
        return command
    except Exception as e:
        return f"# Đã có lỗi xảy ra khi gọi Gemini API: {e}"

# Câu ví dụ phức tạp từ người dùng
complex_sentence = "nhân viên thuộc bộ phận sản xuất phải tham gia khóa huấn luyện an ninh để làm việc trong dự án X"
generated_command = generate_mongodb_command_with_gemini(complex_sentence)

print("Câu đầu vào: ", complex_sentence)
print("\n--- Câu lệnh được tạo bởi Gemini ---")
print(generated_command)

Câu đầu vào:  nhân viên thuộc bộ phận sản xuất phải tham gia khóa huấn luyện an ninh để làm việc trong dự án X

--- Câu lệnh được tạo bởi Gemini ---
db.createRole(
   {
     role: "productionProjectXAccess",
     privileges: [
       {
         resource: { db: "access_control_demo_db", collection: "users" },
         actions: [ "find", "update", "insert", "remove" ],
         applyWhen: {
           "department": "sản xuất",
           "security_training_completed": true,
           "projects": "X"
         }
       }
     ],
     roles: []
   }
)
